In [1]:
import numpy as np

In [2]:
data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

**margin**  
`gamma`

In [3]:
gamma = ((np.dot(th.T, data) + th0)*labels)/np.linalg.norm(th)
gamma

array([[ 0.14142136,  1.41421356, -1.41421356]])

In [4]:
np.sum(gamma)

0.14142135623730967

In [5]:
np.min(gamma)

-1.414213562373095

In [6]:
np.max(gamma)

1.414213562373095

**Hinge loss**

In [7]:
gamma_ref = (2**0.5)/2
gamma_ref

0.7071067811865476

In [8]:
data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

In [9]:
idx = gamma < gamma_ref
idx

array([[ True, False,  True]])

In [10]:
gamma[idx]

array([ 0.14142136, -1.41421356])

In [11]:
L_h = np.zeros((1, 3))
L_h

array([[0., 0., 0.]])

In [12]:
L_h[idx] = 1 - (gamma[idx]/gamma_ref)
L_h

array([[0.8, 0. , 3. ]])

In [13]:
def f1(x):
    return float((2 * x + 3) ** 2)

def df1(x):
    return float(2 * 2 * (2 * x + 3))

def f2(v):
    x = float(v[0, 0])
    y = float(v[1, 0])
    return (x**2) + (y**2)

def df2(v):
    x = float(v[0, 0]); y = float(v[1, 0])
    return np.array([[(2 * x), (2 * y)]]).T

In [14]:
def step_size_fn(i):
    return 0.1


def gd(f, df, x0, step_size_fn, max_iter):
    xs = [x0]
    fs = [f(x0)] 
    for i in range(max_iter):
        x_old = xs[-1]
        x_new = x_old - (step_size_fn(i) * df(x_old))
        xs.append(x_new)
        fs.append(f(x_new))
        if abs(fs[-1] - fs[-2]) < 0.00001:
            break
    return (xs[-1], xs, fs)

In [15]:
x, xs, fs = gd(f2, df2, np.array([[4, 4]]).T, step_size_fn, 400)
xs[2]

array([[2.56],
       [2.56]])

In [16]:
f2(xs[2])

13.1072

In [17]:
v = np.array([[4, 5]]).T
v

array([[4],
       [5]])

In [18]:
df2(v)

array([[ 8.],
       [10.]])

In [19]:
v_1 = v - (0.01 * df2(v))
v_1

array([[3.92],
       [4.9 ]])

In [20]:
v[1, 0]

5

In [21]:
def num_grad(f, delta=0.001):
    def df(x):
        d, _ = x.shape
        delv = np.zeros((d, 1))
        gradv = np.zeros((d, 1))
        for i in range(d):
            delv[i, 0] = delta
            gradv[i, 0] = (f(x + delv) - f(x - delv)) / (2 * delta)
            delv[i, 0] = 0
        return gradv
    return df

In [22]:
num_grad(f1(2), delta=0.001)

<function __main__.num_grad.<locals>.df(x)>

In [23]:
x = np.array([[2.0, 2.0]]).T
ans=(num_grad(f2)(x).tolist(), x.tolist())
ans

([[3.9999999999991154], [3.9999999999991154]], [[2.0], [2.0]])

In [24]:
num_grad(f2)(x)

array([[4.],
       [4.]])

In [25]:
def hinge(v):
    c = 1 - v
    idx = c <= 0
    c[idx] = 0
    return c

In [26]:
def hinge_loss(x, y, th, th0):
    margin = y * (np.dot(th.T, x) + th0)
    return hinge(margin)

In [27]:
hinge_loss(data, labels, th, th0)

array([[0.8, 0. , 3. ]])

In [28]:
a = np.array([[-1, 0.7, 2]]).T
a

array([[-1. ],
       [ 0.7],
       [ 2. ]])

In [29]:
c = 1 - a
idx = c <= 0
idx

array([[False],
       [False],
       [ True]])

In [30]:
c[idx] = 0
c

array([[2. ],
       [0.3],
       [0. ]])

**np.invert() - Invering a numpy boolean array**

In [31]:
np.invert(idx)

array([[ True],
       [ True],
       [False]])

In [32]:
copy = np.copy(c)
copy

array([[2. ],
       [0.3],
       [0. ]])

In [33]:
copy = 1 - copy
copy

array([[-1. ],
       [ 0.7],
       [ 1. ]])

In [34]:
c

array([[2. ],
       [0.3],
       [0. ]])

In [35]:
def d_hinge(v):
    dv = np.copy(v)
    idx = v < 1
    dv[idx] = -1 
    not_idx = np.invert(idx)
    dv[not_idx] = 0
    return dv

In [36]:
d_hinge(np.array([[ 71.]])).tolist()

[[0.0]]

In [37]:
d_hinge(np.array([[ -23.]])).tolist()

[[-1.0]]

In [38]:
d_hinge(np.array([[ 71, -23.]])).tolist()

[[0.0, -1.0]]

In [39]:
def d_hinge_loss_th(x, y, th, th0):
    margin = y * (np.dot(th.T, x) + th0)
    d_hinge_coeff = d_hinge(margin)
    grad_hinge_loss = d_hinge_coeff * (y * x)
    return grad_hinge_loss
    

In [40]:
d_hinge_loss_th(data, labels, th, th0)

array([[-1.1, -0. ,  4. ],
       [-3.1, -0. ,  2. ]])

In [41]:
'''Analytical partial derivative of hinge_loss wrt th0 is 0 or -y.
    First send the margin to the d_hinge function to get the coefficient
    0 or -1(to get the piece-wise behavior). Then multiply it with y to get the derivative'''
# Returns the gradient of hinge_loss(x, y, th, th0) with respect to th0
def d_hinge_loss_th0(x, y, th, th0):
    margin = y * (np.dot(th.T, x) + th0)
    d_hinge_coeff = d_hinge(margin)
    partial_hinge_loss_th0 = d_hinge_coeff * y
    return partial_hinge_loss_th0


In [42]:
d_hinge_loss_th0(data, labels, th, th0)

array([[-1., -0.,  1.]])

In [43]:
np.sum(d_hinge_loss_th(data, labels, th, th0), axis=1)

array([ 2.9, -1.1])

In [44]:
np.array([np.sum(d_hinge_loss_th(data, labels, th, th0), axis=1)]).T / 3

array([[ 0.96666667],
       [-0.36666667]])

In [54]:
def d_svm_obj_th(x, y, th, th0, lam):
    _, n = x.shape
    ''' Adding up all the columns of gradient of hinge loss wrt th into a single column and 
    dividing by n to get the average'''
    avg_grad_hinge_loss_th = np.array([np.sum(d_hinge_loss_th(x, y, th, th0), axis=1)]).T / n
    grad_J_th = avg_grad_hinge_loss_th + (lam * th)
    return grad_J_th

In [55]:
d_svm_obj_th(data, labels, th, th0, 0.01)

array([[ 0.97666667],
       [-0.35666667]])

In [47]:
def d_svm_obj_th0(x, y, th, th0, lam):
    _, n = x.shape
    avg_partial_hinge_loss_th0 = np.array([np.sum(d_hinge_loss_th0(x, y, th, th0), axis=1)]).T / n
    return avg_partial_hinge_loss_th0

In [48]:
d_svm_obj_th0(data, labels, th, th0, 0.001)

array([[0.]])

In [49]:
X1 = np.array([[1, 2, 3, 9, 10]])
y1 = np.array([[1, 1, 1, -1, -1]])
th1, th10 = np.array([[-0.31202807]]), np.array([[1.834     ]])
X2 = np.array([[2, 3, 9, 12],
               [5, 2, 6, 5]])
y2 = np.array([[1, -1, 1, -1]])
th2, th20=np.array([[ -3.,  15.]]).T, np.array([[ 2.]])


In [56]:
d_L_h = d_hinge_loss_th(X2, y2, th2, th20)
d_L_h

array([[ 0.,  3.,  0., 12.],
       [ 0.,  2.,  0.,  5.]])

In [62]:
np.array([np.sum(d_L_h, axis=1)]).T / d_L_h.shape[1] + (0.01 * th2)

array([[3.72],
       [1.9 ]])

In [72]:
np.ones((5, 1))[4:5]

array([[1.]])

In [77]:
def batch_svm_min(data, labels, lam):
    def svm_min_step_size_fn(i):
       return 2/(i+1)**0.5
    # Initializing th i.e. a single vector containing [th, th0].T 
    d, n = data.shape
    th_init = np.zeros((d+1, 1))
    # Applying gradient descent
    ths = [th_init]
    svms = [svm_obj(data, labels, th_init[:d], th_init[d:d+1], lam)]
    for i in range(100):
        th_old = ths[-1]
        th_new = th_old - svm_min_step_size_fn(i) * svm_obj_grad(data, labels, th_old[:d], th_old[d:d+1], lam)
        ths.append(th_new)
        svms.append(svm_obj(data, labels, th_new[:d], th_new[d:d+1], lam))
        if abs(svms[-1] - svms[-2]) < 0.00001:
            break
    return ths[-1], ths, svms
 

In [78]:
def svm_obj(x, y, th, th0, lam):
    _, n = x.shape
    avg_hinge_loss = np.sum(hinge_loss(x, y, th, th0)) / n
    # SVM objective
    J = avg_hinge_loss + (lam * ((np.linalg.norm(th)) ** 2))
    return J


In [79]:
def svm_obj_grad(X, y, th, th0, lam):
    svm_grad_th = d_svm_obj_th(X, y, th, th0, lam)
    svm_grad_th0 = d_svm_obj_th0(X, y, th, th0, lam)
    svm_grad = np.vstack((svm_grad_th, svm_grad_th0))
    return svm_grad


In [80]:
batch_svm_min(X2, y2, 0.001)

(array([[-2.63931935],
        [ 4.54584912],
        [-2.20596531]]),
 [array([[0.],
         [0.],
         [0.]]),
  array([[-2.],
         [ 2.],
         [ 0.]]),
  array([[1.18480894],
         [4.11849192],
         [0.35355339]]),
  array([[-3.14668618],
         [ 2.09301035],
         [-0.22379688]]),
  array([[-0.89353949],
         [ 3.59091734],
         [ 0.02620312]]),
  array([[-4.24684225],
         [ 2.02245794],
         [-0.42101047]]),
  array([[-2.40625741],
         [ 3.24555148],
         [-0.21688633]]),
  array([[-1.27054503],
         [ 3.99902702],
         [-0.21688633]]),
  array([[-3.92129705],
         [ 2.75876242],
         [-0.57043972]]),
  array([[-2.41868285],
         [ 3.75692324],
         [-0.40377305]]),
  array([[-1.46846985],
         [ 4.38700269],
         [-0.40377305]]),
  array([[-3.72891941],
         [ 3.32906752],
         [-0.7052844 ]]),
  array([[-2.42772841],
         [ 4.19317088],
         [-0.56094683]]),
  array([[-2.8424069 